# Extract text from Rick and Morty csv
Text from here https://www.kaggle.com/datasets/andradaolteanu/rickmorty-scripts

In [110]:
!pip install nltk

In [111]:
import numpy as np
import csv
import pandas as pd
import os
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TreebankWordTokenizer

from nltk.tag import pos_tag
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [11]:
df = pd.read_csv('datasets/rick-morty/RickAndMortyScripts.csv')

In [22]:
text_combined = ' '.join(df['line'])

In [23]:
out_fn = 'datasets/rick-morty/RickAndMortyScripts.txt'
with open(out_fn, 'w') as f:
        f.write(text_combined)

In [21]:
os.stat(out_fn)

os.stat_result(st_mode=33188, st_ino=293051736, st_dev=165, st_nlink=1, st_uid=1000, st_gid=100, st_size=134781, st_atime=1678697254, st_mtime=1678697252, st_ctime=1678697252)

In [131]:
# Replace words with tokenization and detokenization
text = text_combined

t = TreebankWordTokenizer()
d = TreebankWordDetokenizer()

toks = t.tokenize(text)
tagged_words = pos_tag(toks)

# Replace with PERSON
#filtered_words = [tag[0] if tag[1] not in ('NNP', 'NN') else "PERSON" for tag in tagged_words]
#filtered_text = d.detokenize(filtered_words)

In [125]:
tagged_words[:10]

[('Morty', 'NN'),
 ('!', '.'),
 ('You', 'PRP'),
 ('got', 'VBD'),
 ('ta', 'JJ'),
 ('come', 'VBN'),
 ('on.', 'RB'),
 ('Jus', 'NNP'),
 ("'", 'POS'),
 ('...', ':')]

In [133]:
ne_tree = ne_chunk(tagged_words)

# TODO Should do it by sentence afterwards, or memory might explode

In [139]:
# Replace PERSON entities with PERSON
# This analyses the text and replaces person entities with the text PERSON

my_tokens = []

def traverse_tree(tree):
    global tokens
    if isinstance(tree, nltk.Tree):
        if tree.label() == 'PERSON':
            my_tokens.append("PERSON")
            #print('PERSON:', ' '.join([child[0] for child in tree]))
        else:
            for child in tree:
                traverse_tree(child)
    else:
        my_tokens.append(tree[0])
        #print('NOT PERSON:', tree[0])

traverse_tree(ne_tree)

#print(my_tokens[:20])
#print(d.detokenize(my_tokens))
clean_text = d.detokenize(my_tokens)


In [141]:
# Do text replacements

# Replace remaining names
clean_text = clean_text.replace("Jessica", "PERSON")
clean_text = clean_text.replace("Morty", "PERSON")


In [142]:
print(clean_text[:300])

PERSON! You gotta come on. Jus'...you gotta come with me. What, PERSON? What’s going on? I got a surprise for you, PERSON. It's the middle of the night. What are you talking about? Come on, I got a surprise for you. Come on, hurry up. Ow! Ow! You're tugging me too hard! We gotta go, gotta get outta 


In [143]:
out_fn = 'datasets/rick-morty/RickAndMortyScripts-cleaned.txt'
with open(out_fn, 'w') as f:
        f.write(clean_text)

In [93]:
#### Example code from chatgpt ####

text = "John Smith is a software engineer at Google. He graduated from MIT in 2010."
#text = text_combined
# tokenize text into sentences
sentences = sent_tokenize(text)

# apply NER to identify named entities
person_names = set()
for sentence in sentences:
    tokens = word_tokenize(sentence)
    ner_tags = nltk.ne_chunk(nltk.pos_tag(tokens))
    person_names.update([chunk.leaves()[0][0] for chunk in ner_tags if hasattr(chunk, 'label') and chunk.label() == 'PERSON'])
    
# remove person names from text
filtered_sentences = []
for sentence in sentences:
    tokens = word_tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in person_names]
    filtered_sentence = ' '.join(filtered_tokens)

    # remove extra spaces before punctuation marks
    filtered_sentence = re.sub(r'\s+([.,\';!?])', r'\1', filtered_sentence)
    
    filtered_sentence = re.sub(r'\s+([.,;!?])', r'\1', filtered_sentence)
    #filtered_sentence = re.sub(r'([.,;!?])\s+', r'\1 ', filtered_sentence)
    #filtered_sentence = re.sub(r'\s*([.,;!?]+)\s*', r'\1 ', filtered_sentence)

    filtered_sentences.append(filtered_sentence)

# join sentences into final text
clean_text = ' '.join(filtered_sentences)

print(clean_text[:300])

KeyboardInterrupt: 

In [103]:

#### EXAMPLE EXTRACT NAMED ENTITIES ####

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag

### WORKING OK
# Load the text you want to extract names from
text = "John Smith and Sarah Johnson went to the park. They met with Alex Kim there."
text = text_combined

# Tokenize the text into sentences and words
sentences = sent_tokenize(text)
words = [word_tokenize(sent) for sent in sentences]

# Tag each word with its part-of-speech (POS)
tagged_words = [pos_tag(sent) for sent in words]

# Extract all named entities (NEs) from the tagged words
named_entities = []
for sent in tagged_words:
    for chunk in nltk.ne_chunk(sent):
        if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
            named_entities.append(' '.join(c[0] for c in chunk))

# Print the list of names
#print(named_entities)